# 'Data_processing.ipynb' is created by Yue on Apr 26, 2023 for data preprocessing.

Workflow:
1. Load original instant data of ux,uy,uz.
2. Filter by missing counts. Add flag when the number of nan exceeds 10%.
4. Despike. Add flags when the number of spikes exceeds 1%.
5. Calculate wind angle. Add flags when the wind angle is within (120,240) degrees.
6. Tilt rotation.
7. Detrend (linear detrending and high-pass filtering).
8. Density correction. (neglect)
9. Spectral correction. (neglect)

Notes:
1. input directory:/save_preprocessed_data.
0. output directory: /save_processed_data.
0. q,P measurements are very weird so I didn't perform de-spiking on them.
0. qc=0: data is good; =1: data is bad.
0. webb_corr=1: correct q,T,C; =2: correct q,C; =0: no correction.
0. Do high-pass filtering on u,v,w,T with the same cutoff wavelength.
0. All the instantaneous variables will be saved in separate files by hours.
0. The hourly variables below will be saved in by days in: /save_processed_data.\
    \['ts_dspk_wind_ang', 'u_filt_size'],
    ['u_nspikes', 'v_nspikes', 'w_nspikes', 'T_nspikes'],
    ['qc_ux_nan', 'qc_uy_nan', 'qc_uz_nan', 'qc_T_nan', 'qc_q_nan', 'qc_P_nan']

# Set up environment

In [2]:
# This jupyter notebook command inserts matplotlib graphics in to the workbook
%matplotlib inline

# import packages
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import os.path
import pickle
import time
from matplotlib.pyplot import figure
import scipy.io as sio
from datetime import date, timedelta
from math import *
from scipy.stats import gmean
from scipy import ndimage
from scipy import stats
from scipy import signal
import seaborn as sns
from scipy.signal import butter,sosfiltfilt,filtfilt
from scipy import fftpack
import time

# Define parameters

In [3]:
# directories
IN_DIR = "/projectnb/urbanclimate/yueqin/idaho_ec_jupyter/save_preprocessed_data/"
OUT_DIR = "/projectnb/urbanclimate/yueqin/idaho_ec_jupyter/save_processed_data/"

# physical constants (or values that are assumed constant)
Rw  = 461.5     # ideal gas constant for water vapor, J/kg*K
Rd  = 287.05    # ideal gas constant for dry air, J/kg*K
Lv  = 1000*2257 # latent heat of vaporization (water), J/kg
Cp  = 1005      # approximate constant pressure specific heat of air, J/kg*K
k   = 0.4      # Von Karman constant
g   = 9.81      # acceleration of gravity, m/s^2

# global constants
sonum    =12                       # number of sonic
z  = np.array([1.2,2,3.5,6,9,12.5,16.5,23,30,40,50,60])  # height of sonic above ground, 
frequency=10                   # sampling rate, Hz
time_avg =3600                  # average time, s
rpat = time_avg*frequency           # number of lines for a loop

# set the range of wind angle accepted
min_wnd  = 120
max_wnd  = 240

# Filter requirements.
T = time_avg         # Sample Period
l_cutoff = 2000      # cutoff wavelength, m
order = 10       # filter order
nyq = 0.5 * frequency  # Nyquist Frequency
n = int(T * frequency) # total number of samples

# input variables
ins_var=['u_ins','v_ins','w_ins','Tsonic_ins','diag_csat_ins','q_ins','P_ins']

# output variables
out_tur = ['ux_dspk', 'uy_dspk', 'uz_dspk', 'T_dspk', 'q_ins_rnan', 'P_ins_rnan',
           'u_dspk_2rot_ldtr', 'v_dspk_2rot_ldtr', 'w_dspk_2rot_ldtr', 'T_dspk_ldtr', 
           'u_dspk_2rot_trend', 'v_dspk_2rot_trend', 'w_dspk_2rot_trend', 'T_dspk_trend',
           'u_dspk_2rot_filt', 'v_dspk_2rot_filt', 'w_dspk_2rot_filt', 'T_dspk_filt']
out_other = ['ts_dspk_wind_ang', 'u_filt_size']
out_qf = ['qc_ux_nan', 'qc_uy_nan', 'qc_uz_nan', 'qc_T_nan','qc_q_nan', 'qc_P_nan',
          'qc_ux_dspk', 'qc_uy_dspk', 'qc_uz_dspk', 'qc_T_dspk','qc_wdir_dspk']
out_nspikes = ['u_nspikes', 'v_nspikes', 'w_nspikes', 'T_nspikes']

# controls
# webb_corr = 2 # do webb-corr on q and C only

# Define functions

In [4]:
def date_list(sdate,edate):
    """method used for creating date list"""
    delta = edate - sdate       # as timedelta
    day = [sdate+timedelta(days=x) for x in range(delta.days+1)]
    return day

def rmnan(data,flag):  
    """ method used for checking & rm nan"""  
    data[flag >= 65] = np.nan ## ！！！！！ get rid of bad data (thershold is 65)! ! ! ! !
    nansum = np.sum(np.isnan(data))
    qc = 0
    if nansum >= time_avg*frequency/10:
        data[:] = np.nan
        qc = 1
    return data,qc
    # for icol in range(sonum):
    #     nansum = np.sum(np.isnan(data[:,icol]),0)
    #     # if there are more than 10% nan in an hour, discard all data at that level
    #     if nansum >= time_avg*frequency/10:
    #         data[:,icol] = np.nan
    # return data
def get_wind_ang(ux,uy,nl):
    """
    !!!arctan return values in radians!!!
    for 1 level only
    calculate mean wind angle on the xy-plane (!! must do before double rotation)
    The CSAT3 (the anemometer arms, tripods) is aligned northward and if u is positive, the wind is northerly. 
    If v is positive, the wind is westerly.
    u:north(+)->south(-), v:west(+)->east(-)
    """
    u_avg = np.nanmean(ux,axis=0) # size = 1
    v_avg = np.nanmean(uy,axis=0)
    rot_ang_v = degrees(np.arctan(v_avg/u_avg))
    # print('nl='+str(nl))
    # if nl==2:
    #     wind_ang = rot_ang_v*np.nan
    #     mask1 = np.logical_and(u_avg>=0, v_avg<=0)
    #     wind_ang[mask1] = -rot_ang_v[mask1] # northeast
    #     mask2 = np.logical_and(u_avg<=0, v_avg<=0)
    #     wind_ang[mask2] = 180-rot_ang_v[mask2] # southest
    #     mask3 = np.logical_and(u_avg<=0, v_avg>=0)
    #     wind_ang[mask3] = 180-rot_ang_v[mask3] # southwest
    #     mask4 = np.logical_and(u_avg>=0, v_avg>=0)
    #     wind_ang[mask4] = 360-rot_ang_v[mask4] # northwest         
    if nl==1:
        wind_ang = rot_ang_v
        if np.logical_and(u_avg>=0, v_avg<=0):
            wind_ang = -rot_ang_v # northeast
        if np.logical_and(u_avg<=0, v_avg<=0):
            wind_ang = 180-rot_ang_v # southest
        if np.logical_and(u_avg<=0, v_avg>=0):
            wind_ang = 180-rot_ang_v # southwest
        if np.logical_and(u_avg>=0, v_avg>=0):
            wind_ang = 360-rot_ang_v # northwest 
    return wind_ang

def wind_ang(ux,uy):
    """
    calculate mean wind angle on the xy-plane (!! must do before double rotation)
    The CSAT3 (the anemometer arms, tripods) is aligned northward and if u is positive, the wind is northerly. 
    If v is positive, the wind is westerly.
    u:north(+)->south(-), v:west(+)->east(-)
    """
    u_avg = np.nanmean(ux,axis=0) # size = 12
    v_avg = np.nanmean(uy,axis=0)
    rot_ang_v = np.arctan(v_avg/u_avg)
    rot_ang_v = rot_ang_v*360/2/math.pi
    mask1 = np.logical_and(u_avg>=0, v_avg<=0)
    rot_ang_v[mask1] = -rot_ang_v[mask1] # northeast
    mask2 = np.logical_and(u_avg<=0, v_avg<=0)
    rot_ang_v[mask2] = 180-rot_ang_v[mask2] # southest
    mask3 = np.logical_and(u_avg<=0, v_avg>=0)
    rot_ang_v[mask3] = 180-rot_ang_v[mask3] # southwest
    mask4 = np.logical_and(u_avg>=0, v_avg>=0)
    rot_ang_v[mask4] = 360-rot_ang_v[mask4] # northwest
    
    # quality control: wind angle>120 and < 240
    qc = np.zeros(12)
    msk = np.logical_and(rot_ang_v> min_wnd,rot_ang_v<max_wnd)
    qc[msk] = 1
    return rot_ang_v,qc

def double_rot(ux,uy,uz):
    """
    Double rotation method (Note yaw correction must perform before pitch correction)
    https://www.licor.com/env/support/EddyPro/topics/anemometer-tilt-correction.html#:~:
    text=Double%20rotation%20method,by%20the%20flux%20averaging%20length.
    # such that the hourly avg of v and w will be zero
    # only u,v,w will be rotated, other variables remain the same
    """
    u_avg = np.nanmean(ux,axis=0) # size = 12
    v_avg = np.nanmean(uy,axis=0)
    w_avg = np.nanmean(uz,axis=0)
    # 1) yaw rotation (to remove v component)
    C1 = (u_avg**2 + v_avg**2) ** 0.5 # Magnitude of horizontal wind vector
    rot_mat_1 = u_avg/C1 # cos(theta)
    rot_mat_2 = v_avg/C1 # sin(theta)
    rot_mat_3 = -v_avg/C1
    rot_mat_4 = u_avg/C1
    u_rot = ux * rot_mat_1 + uy*rot_mat_2
    v_rot = ux * rot_mat_3 + uy*rot_mat_4
    u_ins_yawrot = u_rot # Intermediate rotated u after yaw correction
    v_ins_2rot = v_rot
    u_avg_yawrot = np.nanmean(u_ins_yawrot,axis=0) # Recompute u mean after yaw rotation
    # v_avg_2rot = np.nanmean(v_ins_2rot,axis=0)

    # 2) pitch rotation (to remove w component)
    C2 = (u_avg_yawrot**2 + w_avg**2) ** 0.5 # Magnitude of u-w plane vector
    rot_mat_1 = u_avg_yawrot/C2
    rot_mat_2 = w_avg/C2  # cos(phi)
    rot_mat_3 =-w_avg/C2  # sin(phi)
    rot_mat_4 = u_avg_yawrot/C2
    u_rot = u_ins_yawrot * rot_mat_1 + uz*rot_mat_2
    w_rot = u_ins_yawrot * rot_mat_3 + uz*rot_mat_4
    u_ins_2rot = u_rot
    w_ins_2rot = w_rot
    # u_avg_2rot = np.nanmean(u_ins_2rot,axis=0)
    # w_avg_2rot = np.nanmean(w_ins_2rot,axis=0)
    
    return u_ins_2rot,v_ins_2rot,w_ins_2rot

def butter_lowpass_filter(filt_type, data, cutoff, fs, order):
    """
    The frequency response of the Butterworth filter is maximally flat 
    (i.e. has no ripples) in the passband and rolls off towards zero in the stopband, 
    hence its one of the most popular low pass filter.
    
    data shoule be turbulent component!
    """
    # replace nan by mean value
    data[np.argwhere(np.isnan(data))] = np.nanmean(data)
    normal_cutoff = cutoff / nyq
    # Get the filter coefficients 
    sos = butter(order, normal_cutoff, btype=filt_type, analog=False, output='sos',fs=fs)
    # Return the filtered output with the same shape as data
    # The function sosfiltfilt should be preferred over filtfilt 
    # for most filtering tasks, as second-order sections have fewer numerical problems.
    y = sosfiltfilt(sos, data) 
    return y

def get_u_star(u_tur_in,v_tur_in,w_tur_in):
    uw = np.nanmean(u_tur_in*w_tur_in,axis=0)
    vw = np.nanmean(v_tur_in*w_tur_in,axis=0)
    u_star = (np.maximum(0,(uw**2+vw**2)**0.5))**0.5
    return u_star
  

def dtrd(data):
    # Only return the turbulent component  
    # The result is equal to signal.detrend(data)+np.mean(data)
    ct = np.arange(len(data))
    a = -(len(data)*np.nansum(ct*data, axis=0) - np.nansum(ct, axis=0) * 
          np.nansum(data, axis=0)) / (np.nansum(ct**2, axis=0)-(np.nansum(ct, axis=0))**2)
    b = (np.nansum(data, axis=0) - a * np.nansum(ct, axis=0))/len(data)
    data_dtr = data + (a*ct+b) - np.nanmean(data)
    return data_dtr

def get_ist(data):
    """
    Return the non-stationarity index for every hourly time series 
    """
    data_5min = data.reshape([12,-1]) # split each hour into 12 chunks/every 5 min
    cvm = np.nanmean(np.nanvar(data_5min,axis=1)) # avg of the variance of each chunck
    ist_5min = abs(cvm-np.nanvar(data))/np.nanvar(data)
    return ist_5min

def z_score(intensity):
    """
    Z-score based approach for spike detection
    """
    mean_int = np.nanmean(intensity)
    std_int = np.nanstd(intensity)
    z_scores = (intensity-mean_int) / std_int
    return z_scores

def fixer_backup(y,thres):
    """
    remove spikes and fix them with the mean of its immediate neighbors.
    Following Vickers and Mahrt (1997), 
    https://www.licor.com/env/support/EddyPro/topics/despiking-raw-statistical-screening.html
    """
    qc = 0
    it = 0
    y_original = y.copy()
    while it < 20: # iterate 20 times
        # print(f"the {it} iterations:")
        # print(f"the 0 windows")
        y_sub = y[0:12000] # moving window is 20 min
        y_fix = y_sub.copy()
        spikes = abs(np.array(z_score(y_sub))) > thres
        n_con_spk = 0 # counts of more than 4 consecutive spikes
        for i in np.where(spikes != 0)[0]:  # If we have an spike in position i
            if i == 12000 - 1:
                w2 = np.arange(i-3,i+1)
                w1 = w2
            elif i == 12000 - 2:
                w2 = np.arange(i-4,i)
                w1 = np.arange(i-3,i+1)
            elif i == 12000 - 3:
                w2 = np.arange(i-5,i-1)
                w1 = np.arange(i-3,i+1)
            else:
                w1 = np.arange(i-3,i+1)
                w2 = np.arange(i,i+4)
            # 4 consecutive outliers
            # are considered as a local trend and not counted as a spike. 
            if np.sum(spikes[w1])==4 or np.sum(spikes[w2])==4:
                # print(f"4 consecutive spikes.")
                n_con_spk += 1
            else:
                if i == 12000 - 1:
                    w = np.arange(12000-3,12000)
                else:
                    w = np.arange(i-1,i+2) # we select 3 points around our spike
                ww = w[spikes[w] == 0] # From such interval, we choose the ones which are not spikes
                y_fix[i] = np.mean(y_sub[ww]) # and we average their values
        # nspikes = np.nansum(spikes) - n_con_spk
        y_new = y_fix
        # 2nd to 6th moving window
        for iw in np.arange(1,5):
            n_con_spk = 0
            # print(f"the {iw} windows")
            y_sub = y[iw*6000:iw*6000+12000]
            y_fix = y_sub.copy()
            spikes = abs(np.array(z_score(y_sub))) > thres
            for i in np.where(spikes != 0)[0]:  # If we have an spike in position i
                if i >= 6000:
                    if i == 12000 - 1:
                        w2 = np.arange(i-3,i+1)
                        w1 = w2
                    elif i == 12000 - 2:
                        w2 = np.arange(i-4,i)
                        w1 = np.arange(i-3,i+1)
                    elif i == 12000 - 3:
                        w2 = np.arange(i-5,i-1)
                        w1 = np.arange(i-3,i+1)
                    else:
                        w1 = np.arange(i-3,i+1)
                        w2 = np.arange(i,i+4)
                    # 4 consecutive outliers
                    # are considered as a local trend and not counted as a spike. 
                    if np.sum(spikes[w1])==4 or np.sum(spikes[w2])==4:         
                        # print(f"4 consecutive spikes.")
                        n_con_spk += 1
                    else:
                        if i == 12000 - 1:
                            w = np.arange(12000-3,12000)
                        else:
                            w = np.arange(i-1,i+2) # we select 3 points around our spike
                        ww = w[spikes[w] == 0] # From such interval, we choose the ones which are not spikes
                        y_fix[i] = np.mean(y_sub[ww]) # and we average their values
            nspk = np.nansum(spikes[6000:12000]) - n_con_spk
            y_new = np.append(y_new,y_fix[6000:12000])
            # nspikes += nspk
        # print(f"{nspikes} spikes")
        # print("-----------")        
        # if it == 0:
        #     n_spikes = nspikes
        # else:
        n_spikes = np.sum((y_new-y_original)!=0)
        if n_spikes > 0.01*rpat: # accepted spikes is 1%
            qc = 1 # quality flag = 1, should be discarded from the results dataset.
            print("Too much number of spikes")
            break
        if n_spikes == 0:
            break
        y = y_new
        it += 1
    return y_new,qc,n_spikes

def fixer(y,thres):
    """
    remove spikes and fix them with the mean of its immediate neighbors.
    Following Vickers and Mahrt (1997), 
    https://www.licor.com/env/support/EddyPro/topics/despiking-raw-statistical-screening.html
    """
    qc = 0
    it = 0
    y_original = y.copy()
    while it < 20: # iterate 20 times
        # print(f"the {it} iterations:")
        # print(f"the 0 windows")
        y_sub = y[0:12000] # moving window is 20 min
        y_fix = y_sub.copy()
        spikes = abs(np.array(z_score(y_sub))) > thres
        n_con_spk = 0 # counts of more than 4 consecutive spikes
        for i in np.where(spikes)[0]:  # If we have an spike in position i
            if i == 12000 - 1:
                w2 = np.arange(i-3,i+1)
                w1 = w2
            elif i == 12000 - 2:
                w2 = np.arange(i-4,i)
                w1 = np.arange(i-3,i+1)
            elif i == 12000 - 3:
                w2 = np.arange(i-5,i-1)
                w1 = np.arange(i-3,i+1)
            else:
                w1 = np.arange(i-3,i+1)
                w2 = np.arange(i,i+4)
            # 4 consecutive outliers
            # are considered as a local trend and not counted as a spike. 
            if np.sum(spikes[w1])==4 or np.sum(spikes[w2])==4:
                # print(f"4 consecutive spikes.")
                n_con_spk += 1
            else:
                if i == 12000 - 1:
                    w = np.arange(12000-3,12000)
                else:
                    w = np.arange(i-1,i+2) # we select 3 points around our spike
                ww = w[spikes[w] == 0] # From such interval, we choose the ones which are not spikes
                y_fix[i] = np.mean(y_sub[ww]) # and we average their values
        # nspikes = np.nansum(spikes) - n_con_spk
        y_new = y_fix
        # 2nd to 6th moving window
        for iw in np.arange(1,5):
            n_con_spk = 0
            # print(f"the {iw} windows")
            y_sub = y[iw*6000:iw*6000+12000]
            y_fix = y_sub.copy()
            spikes = abs(np.array(z_score(y_sub))) > thres
            for i in np.where(spikes)[0]:  # If we have an spike in position i
                if i >= 6000:
                    if i == 12000 - 1:
                        w2 = np.arange(i-3,i+1)
                        w1 = w2
                    elif i == 12000 - 2:
                        w2 = np.arange(i-4,i)
                        w1 = np.arange(i-3,i+1)
                    elif i == 12000 - 3:
                        w2 = np.arange(i-5,i-1)
                        w1 = np.arange(i-3,i+1)
                    else:
                        w1 = np.arange(i-3,i+1)
                        w2 = np.arange(i,i+4)
                    # 4 consecutive outliers
                    # are considered as a local trend and not counted as a spike. 
                    if np.sum(spikes[w1])==4 or np.sum(spikes[w2])==4:         
                        # print(f"4 consecutive spikes.")
                        n_con_spk += 1
                    else:
                        if i == 12000 - 1:
                            w = np.arange(12000-3,12000)
                        else:
                            w = np.arange(i-1,i+2) # we select 3 points around our spike
                        ww = w[spikes[w] == 0] # From such interval, we choose the ones which are not spikes
                        y_fix[i] = np.mean(y_sub[ww]) # and we average their values
            nspk = np.nansum(spikes[6000:12000]) - n_con_spk
            y_new = np.append(y_new,y_fix[6000:12000])
            # nspikes += nspk
        # print(f"{nspikes} spikes")
        # print("-----------")        
        # if it == 0:
        #     n_spikes = nspikes
        # else:
        n_spikes = np.sum((y_new-y_original)!=0)
        if n_spikes > 0.01*rpat: # accepted spikes is 1%
            qc = 1 # quality flag = 1, should be discarded from the results dataset.
            print("Too much number of spikes")
            break
        if n_spikes == 0:
            break
        y = y_new
        it += 1
    return y_new,qc,n_spikes


def abs_lim(y,lim):
    """
    After de-spiking,
    replace a value that is outside a user-defined plausible range 
    by the mean of neiboring variables. 
    """
    out_lier = abs(y)>lim
    # print(np.sum(out_lier))
    y_lim = y.copy()
    for i in np.where(out_lier!=0)[0]:
        w = np.arange(i-2,i+3) # select 5 points around 
        w2 = w[out_lier[w] == 0] # From such interval, we choose the ones which are not outliers
        y_lim[i] = np.mean(y[w2]) # and we average their values
    return y_lim

def CheckForLess(list1, val): 
    # traverse in the list
    for x in list1: 
        # compare with all the
        # values with value
        if val <= x:
            return False
    return True

# Load data and do the processing

In [5]:
# set up time period and initialize variables
Sdate = date(2020,9,25)
# Sdate = date(2020,10,17)
Edate = date(2021,4,23)
# Edate = date(2020,9,26)
ds = date_list(Sdate,Edate)
write_results = 1
OUT_DIR

'/projectnb/urbanclimate/yueqin/idaho_ec_jupyter/save_processed_data/'

In [ ]:
%%time
for day in ds:
    strday = str(day.strftime("%Y%m%d"))
    fp_stats = IN_DIR + 'u_ins_' + strday +'.pkl'
    if (not os.path.isfile(fp_stats)):
        print(day.strftime("%Y%m%d")+' do not exist')
        continue
    print('start processing:'+ strday)
    # load data    
    for var in ins_var:
    # for var in ['q_ins','P_ins','diag_csat_ins']:
        a_file = open(IN_DIR + var + '_' + strday +'.pkl', "rb")
        globals()[var] = pickle.load(a_file)
        a_file.close()
    u_ins3d = u_ins.reshape(-1,rpat,sonum)
    v_ins3d = v_ins.reshape(-1,rpat,sonum)
    w_ins3d = w_ins.reshape(-1,rpat,sonum)
    T_ins3d = Tsonic_ins.reshape(-1,rpat,sonum)
    q_ins3d = q_ins.reshape(-1,rpat,sonum)
    P_ins3d = P_ins.reshape(-1,rpat,sonum)
    diag_ins3d = diag_csat_ins.reshape(-1,rpat,sonum)
  
    # initialize avg data every day --------------------------
    ## initialize qaulity flag to be 0
    qc_ux_nan = np.zeros((24,sonum)) 
    qc_uy_nan = np.zeros((24,sonum))
    qc_uz_nan = np.zeros((24,sonum))
    qc_T_nan = np.zeros((24,sonum))
    qc_q_nan = np.zeros((24,sonum))
    qc_P_nan = np.zeros((24,sonum))
    
    qc_ux_dspk = np.zeros((24,sonum)) 
    qc_uy_dspk = np.zeros((24,sonum))
    qc_uz_dspk = np.zeros((24,sonum))
    qc_T_dspk = np.zeros((24,sonum))
    
    ## number of spikes
    u_nspikes = np.zeros((24,sonum)) * np.nan
    v_nspikes = np.zeros((24,sonum)) * np.nan
    w_nspikes = np.zeros((24,sonum)) * np.nan
    T_nspikes = np.zeros((24,sonum)) * np.nan
    
    qc_wdir_dspk = np.zeros((24,sonum))
    
    ## hourly averaged wind angle
    ts_dspk_wind_ang = np.zeros((24,sonum)) * np.nan 
    ## high-pass filter size
    u_filt_size = np.zeros((24,sonum)) * np.nan  
    
    ##-------------------------------------------------
    # start the loop over hours
    for ih in range(24): 
        ux_ts = u_ins3d[ih,:,:] #36000*12
        uy_ts = v_ins3d[ih,:,:]
        uz_ts = w_ins3d[ih,:,:]
        T_ts = T_ins3d[ih,:,:]
        q_ts = q_ins3d[ih,:,:]
        P_ts = P_ins3d[ih,:,:]
        diag = diag_ins3d[ih,:,:]
        
        #initialize tur data every hour
        ux_dspk = np.zeros((rpat,sonum)) * np.nan
        uy_dspk = np.zeros((rpat,sonum)) * np.nan
        uz_dspk = np.zeros((rpat,sonum)) * np.nan
        T_dspk = np.zeros((rpat,sonum)) * np.nan

        u_dspk_2rot_ldtr = np.zeros((rpat,sonum)) * np.nan
        v_dspk_2rot_ldtr = np.zeros((rpat,sonum)) * np.nan
        w_dspk_2rot_ldtr = np.zeros((rpat,sonum)) * np.nan
        T_dspk_ldtr = np.zeros((rpat,sonum)) * np.nan

        u_dspk_2rot_trend = np.zeros((rpat,sonum)) * np.nan
        v_dspk_2rot_trend = np.zeros((rpat,sonum)) * np.nan
        w_dspk_2rot_trend = np.zeros((rpat,sonum)) * np.nan
        T_dspk_trend = np.zeros((rpat,sonum)) * np.nan

        u_dspk_2rot_filt = np.zeros((rpat,sonum)) * np.nan
        v_dspk_2rot_filt = np.zeros((rpat,sonum)) * np.nan
        w_dspk_2rot_filt = np.zeros((rpat,sonum)) * np.nan
        T_dspk_filt = np.zeros((rpat,sonum)) * np.nan
        q_ins_rnan = np.zeros((rpat,sonum)) * np.nan
        P_ins_rnan = np.zeros((rpat,sonum)) * np.nan
    
        # Step 1: remove nan
        # start_time = time.time()
        # replace the whole chunk by nan if nan exceeds 10% in an hour
        # Get rid of bad data with diag_csat >= 65
        for il in range(12): # loop over levels
            ux_ts[:,il],qc_ux_nan[ih,il] = rmnan(ux_ts[:,il],diag[:,il])
            uy_ts[:,il],qc_uy_nan[ih,il] = rmnan(uy_ts[:,il],diag[:,il])
            uz_ts[:,il],qc_uz_nan[ih,il] = rmnan(uz_ts[:,il],diag[:,il])
            T_ts[:,il],qc_T_nan[ih,il] = rmnan(T_ts[:,il],diag[:,il])
            q_ins_rnan[:,il],qc_q_nan[ih,il] = rmnan(q_ts[:,il],diag[:,il])
            P_ins_rnan[:,il],qc_P_nan[ih,il] = rmnan(P_ts[:,il],diag[:,il])
        # print('rmnan done')
        
        # Step 2: De-spiking
        # start_time = time.time()
        for il in range(12): # loop over levels
            # if whole chunck is nan then skip de-spiking
            if np.any([qc_ux_nan[ih,il],qc_uy_nan[ih,il],qc_uz_nan[ih,il],qc_T_nan[ih,il]]):
                continue
            else:
                ux_dspk[:,il],qc_ux_dspk[ih,il],u_nspikes[ih,il] = fixer(ux_ts[:,il],thres=3.5) # 36000*12, 24*12, 24*12
                uy_dspk[:,il],qc_uy_dspk[ih,il],v_nspikes[ih,il] = fixer(uy_ts[:,il],thres=3.5)
                uz_dspk[:,il],qc_uz_dspk[ih,il],w_nspikes[ih,il] = fixer(uz_ts[:,il],thres=5)
                T_dspk[:,il],qc_T_dspk[ih,il],T_nspikes[ih,il] = fixer(T_ts[:,il],thres=3.5)
        # print('despike done')  
        # end_time = time.time()
        # duration = end_time - start_time
        # print(f"Time taken for dspk: {duration} seconds")
        
        # Step 3: Calculte wind angle on the xy-plane 
        for il in range(12):
            ts_dspk_wind_ang[ih,il] = get_wind_ang(ux_dspk[:,il],uy_dspk[:,il],1)
            if np.logical_and(ts_dspk_wind_ang[ih,il]> min_wnd,ts_dspk_wind_ang[ih,il]<max_wnd):
                qc_wdir_dspk[ih,il] = 1
        # ts_dspk_wind_ang[ih,:],qc_wdir_dspk[ih,:] = wind_ang(ux_dspk[ih,:,:],uy_dspk[ih,:,:]) # 1*12, 1*12
        
        # Step 4: Double rotation
        # start_time = time.time()
        u_dspk_2rot,v_dspk_2rot,w_dspk_2rot = double_rot(ux_dspk,uy_dspk,uz_dspk)
        # print('2rot done')
        # end_time = time.time()
        # duration = end_time - start_time
        # print(f"Time taken for double rotation: {duration} seconds")
        
        # Calculate mean variables
        u_avg_dspk_2rot = np.nanmean(u_dspk_2rot,axis=0) # 12*1
        v_avg_dspk_2rot = np.nanmean(v_dspk_2rot,axis=0)
        w_avg_dspk_2rot = np.nanmean(w_dspk_2rot,axis=0)
        T_avg_dspk = np.nanmean(T_dspk[:,:],axis=0)
          
        # Step 5: Detrend
        # start_time = time.time()
        for il in range(12): # loop over levels
            # if whole chunck is nan then skip detrend
            if np.any([qc_ux_nan[ih,il],qc_uy_nan[ih,il],qc_uz_nan[ih,il],qc_T_nan[ih,il]]):
                continue
            else:
                ## 4.1 Linear detrend
                u_dspk_2rot_ldtr[:,il] = dtrd(u_dspk_2rot[:,il])
                v_dspk_2rot_ldtr[:,il] = dtrd(v_dspk_2rot[:,il])
                w_dspk_2rot_ldtr[:,il] = dtrd(w_dspk_2rot[:,il])
                T_dspk_ldtr[:,il] = dtrd(T_dspk[:,il])

                ## 4.2 high-pass filter detrend
                u_cutoff = u_avg_dspk_2rot[il]/l_cutoff     # desired cutoff frequency of the filter, Hz
                # print(cutoff)
                u_filt_size[ih,il] = int(1/u_cutoff) # seconds
                # print(filt_size)
                u_tur = u_dspk_2rot[:,il]-u_avg_dspk_2rot[il]
                u_dspk_2rot_trend[:,il] = butter_lowpass_filter('low', u_tur, u_cutoff, frequency, order)
                u_dspk_2rot_filt[:,il] = u_dspk_2rot[:,il]-u_dspk_2rot_trend[:,il]
                # print('1')
                
                v_tur = v_dspk_2rot[:,il]-v_avg_dspk_2rot[il]
                v_dspk_2rot_trend[:,il] = butter_lowpass_filter('low', v_tur, u_cutoff, frequency, order)
                v_dspk_2rot_filt[:,il] = v_dspk_2rot[:,il]-v_dspk_2rot_trend[:,il]
                # print('2')
                
                w_tur = w_dspk_2rot[:,il]-w_avg_dspk_2rot[il]
                w_dspk_2rot_trend[:,il] = butter_lowpass_filter('low', w_tur, u_cutoff, frequency, order)
                w_dspk_2rot_filt[:,il] = w_dspk_2rot[:,il]-w_dspk_2rot_trend[:,il]
                # print('3')
                
                T_tur = T_dspk[:,il]-T_avg_dspk[il]
                T_dspk_trend[:,il] = butter_lowpass_filter('low', T_tur, u_cutoff, frequency, order)
                T_dspk_filt[:,il] = T_dspk[:,il]-T_dspk_trend[:,il]
                
        # end_time = time.time()
        # duration = end_time - start_time
        # print(f"Time taken for detrending: {duration} seconds")  
        # print('detrend done') 
        
        ### end of the hour loop
        if write_results:
            # print(f"start writing {ih}hour")    
            for var_name in out_tur: # write output by hours
            # for var_name in ['q_ins_rnan','P_ins_rnan']: # write output by hours
                # Access the variable by name using globals() - this allows you to get the variable's value by its name as a string
                var_value = globals()[var_name]
                # Construct the filename using the variable's name and the specified date, then save the array to a .npy file
                filename = f"{var_name}_{strday}_{ih:02}00.npy"
                np.save(OUT_DIR + filename, var_value)
    ## end of the day loop
    if write_results: # write output by days
        for var_name in out_other+out_qf+out_nspikes:
        # for var_name in ['qc_q_nan','qc_P_nan']:
            var_value = globals()[var_name]
            filename = f"{var_name}_{strday}.npy"
            np.save(OUT_DIR + filename, var_value)
    
    # print('finish:'+ strday)

In [ ]:
%whos ndarray

# Check results

In [7]:
day1 = '20200925'
dir1 = '/projectnb/urbanclimate/yueqin/idaho_ec_jupyter/processed_data_020724/'
a_file = open(dir1 + 'ts_dspk_wind_ang_' + day1 +'.pkl', "rb")
u_std_filt1 = pickle.load(a_file)
a_file.close()

u_std_filt2 = np.load(OUT_DIR + 'ts_dspk_wind_ang_' + day1 +'.npy')
u_std_filt1-u_std_filt2

FileNotFoundError: [Errno 2] No such file or directory: '/projectnb/urbanclimate/yueqin/idaho_ec_jupyter/processed_data_020724/ts_dspk_wind_ang_20200925.pkl'

In [8]:
for var in ins_var:
    # for var in ['q_ins','P_ins','diag_csat_ins']:
    a_file = open(IN_DIR + var + '_20210423.pkl', "rb")
    globals()[var] = pickle.load(a_file)
    a_file.close()
u_ins3d = u_ins.reshape(-1,rpat,sonum)
v_ins3d = v_ins.reshape(-1,rpat,sonum)
w_ins3d = w_ins.reshape(-1,rpat,sonum)
T_ins3d = Tsonic_ins.reshape(-1,rpat,sonum)
q_ins3d = q_ins.reshape(-1,rpat,sonum)
P_ins3d = P_ins.reshape(-1,rpat,sonum)
diag_ins3d = diag_csat_ins.reshape(-1,rpat,sonum)

In [ ]:
q_ins3d[-9,:,:]

# Example: plot results

In [ ]:
fig = plt.figure(figsize=(10, 4), dpi=150,tight_layout=True)

In [ ]:
fig = plt.figure(figsize=(10, 4), dpi=150,tight_layout=True)
ax1 = fig.add_subplot(121) 
plt.plot(u_std_filt[0,list_sel_m2])
# plt.ylim(0,1.2)
# plt.plot(u_star_ldtr)
ax2 = fig.add_subplot(122) 
plt.plot(u_std_ldtr[0,list_sel_m2])
# plt.ylim(0,1.2)

In [ ]:
fig = plt.figure(figsize=(8, 4), dpi=150,tight_layout=True)
plt.plot(L_H2_filt[:,1])
plt.plot(L_H2_ldtr[:,1])

In [ ]:
fig = plt.figure(figsize=(8, 4), dpi=150,tight_layout=True)
# plt.plot(u_2rot_dspk_filt[:,0])
plt.plot(u_dspk_2rot_ldtr[:,0])
plt.plot(u_dspk_2rot_filt[:,0])